# Battle of the Neighborhoods - Part 1

 ## I. Description of the problem and a discussion of the background.

### Introduction
I would like to open a restaurant in Tulsa, OK. 
This project will be intersing for all who would like to open a new restaurant in an restaurant-hotspot in Tulsa

### Data
But First of all we need to collect Data of all restaurants including their name, id, location (address, latitude, longitude) then pick up the "hot" neighborhood which locates most of the venue. In order to asset Data we use FourSquare and apply folium for visualizing a particular neighbor in which that we will observe customer "traffic" and predict an appropriate location of new restaurant in town.


In [5]:
# for performing your HTTP requests
import requests  
# for xml & html scrapping 
from bs4 import BeautifulSoup 
# for table analysis
import pandas as pd
# write to csv
import csv
# Time
import time
#Visuals
import matplotlib.pyplot as plt
import types
from botocore.client import Config
import ibm_boto3
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium

     |████████████████████████████████| 93 kB 3.4 MB/s  eta 0:00:01


First of all, I need to identify the geographical coordinates of Tulsa

In [87]:
address = 'Tulsa , Oklahoma'

geolocator = Nominatim(user_agent="tulsa_explorer")
location = geolocator.geocode(address)
TUlatitude = location.latitude -0.0005
TUlongitude = location.longitude +0.0005
print('The geograpical coordinate of Tulsa are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tulsa are 36.1556805, -95.9929113.


Then I create a map of the city

In [88]:
# create map of Toronto using latitude and longitude values
map_tulsa = folium.Map(location=[TUlatitude, TUlongitude], zoom_start=10)

map_tulsa

In [89]:
CLIENT_ID = '1Q3YUWN3SWPAOFQECRRUANTEORFNAB1CWJOWST5PTOF0GOKK' # your Foursquare ID
CLIENT_SECRET = 'FZHEZJD1X2ZKKBK4VMZE4MFHVJRI0Y5J10OFR4EQZF0WXBHX' # your Foursquare Secret
VERSION = '20201223' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1Q3YUWN3SWPAOFQECRRUANTEORFNAB1CWJOWST5PTOF0GOKK
CLIENT_SECRET:FZHEZJD1X2ZKKBK4VMZE4MFHVJRI0Y5J10OFR4EQZF0WXBHX


In the next step I collect Data from Foursquare with restaurants in Tulsa; I use a while loop to cover as many parts of the city as possible; Sadly I've exceeded the number of calls per day for today

In [123]:
from pandas.io.json import json_normalize
address = 'Tulsa , Oklahoma'
nearby_venues_total = []
geolocator = Nominatim(user_agent="tulsa_explorer")
location = geolocator.geocode(address)
TUlatitude = location.latitude -1
TUlongitude = location.longitude + 1
print('The geograpical coordinate of Tulsa are {}, {}.'.format(TUlatitude, TUlongitude))


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

lo = 0
la = 0
LIMIT = 100
radius = 500
while TUlongitude > -95.9924113: 
    while TUlatitude < 36.1551805:
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        TUlatitude, 
        TUlongitude, 
        radius, 
        LIMIT)

        results = requests.get(url).json()
        
        #venues = results['response']['groups'][0]['items']
    
        #nearby_venues = json_normalize(venues) # flatten JSON

        #filter columns
        #filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        
       # nearby_venues =nearby_venues.loc[:, filtered_columns]

       # filter the category for each row
        #nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

        #clean columns
        #nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

        #nearby_venues_total.append(nearby_venues)
        
        TUlatitude += 0.1
    TUlongitude -= 0.1
#nearby_venues_total.head()

results

The geograpical coordinate of Tulsa are 35.1556805, -94.9929113.


{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5fe480b06ef73c60c67e8a76'},
 'response': {}}

Therefore I'm not able to proceed with real data; I leave you my code here for my rating; At first I have to drop all duplicates

In [118]:
nearby_venues_total.drop_duplicates(subset ="name", 
                     keep = False, inplace = True) 

AttributeError: 'list' object has no attribute 'drop_duplicates'

In [119]:
print('{} venues were returned by Foursquare.'.format(nearby_venues_total.shape[0]))

AttributeError: 'list' object has no attribute 'shape'

In [59]:
print('There are {} uniques categories.'.format(len(nearby_venues_total['categories'].unique())))

There are 57 uniques categories.


In [60]:
# this is the reslut of my first attempt with only on search for locations in downtown tulsa

# one hot encoding
tulsa_onehot = pd.get_dummies(nearby_venues_total[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tulsa_onehot['Name'] = nearby_venues_total['name'] 

# move neighborhood column to the first column
fixed_columns = [tulsa_onehot.columns[-1]] + list(tulsa_onehot.columns[:-1])
tulsa_onehot = tulsa_onehot[fixed_columns]

tulsa_onehot.head()

,Name,American Restaurant,Art Gallery,Art Museum,BBQ Joint,Bakery,Bar,Baseball Stadium,Beer Garden,Bookstore,...,Shopping Plaza,Social Club,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Theater,Tiki Bar,Vietnamese Restaurant
0,Sisserou's,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guthrie Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Tulsa Performing Arts Center,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Valkyrie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Prhyme,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [61]:
tulsa_grouped = tulsa_onehot.groupby('Name').mean().reset_index()
tulsa_grouped

,Name,American Restaurant,Art Gallery,Art Museum,BBQ Joint,Bakery,Bar,Baseball Stadium,Beer Garden,Bookstore,...,Shopping Plaza,Social Club,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Theater,Tiki Bar,Vietnamese Restaurant
0,108 Contemporary,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Albert G's Bar-B-Q,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,All About Cha,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Amelia's,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Andolini's Pizzeria Cherry Street,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Valkyrie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,Woodward Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,Woody Guthrie Center,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,YMCA Downtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
from folium import plugins


map_Tulsa = folium.Map(ocation=[TUlatitude, TUlongitude], zoom_start=10)

def add_markers(nearby_venues):
    for (j, row) in nearby_venues.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Tulsa)

add_markers(nearby_venues)
hm_data = nearby_venues[["lat", "lng"]].to_numpy().tolist()
map_Tulsa.add_child(plugins.HeatMap(hm_data))

map_Tulsa


### Discussion
As you can see there is a hotspot in Tulsa Arts Districts. But this Data only represents the area of Downtown Tulsa. As you can see there are serveral smaller clusters. One big at Tulsa Arts Districts, and smaller one at the blue dome, oil capital histroical district and at cherry street

### Conclusion

As there are already lots of restaurants in the Area of the Arts Districts, there might be a large competition. The two other hotspots in the area of Downtown Tulsa have many hotels but are close to the big hotspot. All in all I would recoommend to open a restaurant at cherry street where already are some re